In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import IQA_Interpolator from iqa_interpolator

In [ ]:
reports = []
for i in range(1,9):
    reports.append(pd.read_excel(f'relatorios/relatorio{i}.xlsx', decimal=','))
df = pd.concat(reports)
df['Data Coleta'] = pd.to_datetime(df['Data Coleta'], format='%d/%m/%Y')
df.head()

In [ ]:
print(df.columns)
print(df.Parametro.value_counts())

In [ ]:
droppers = ['Cod_Interaguas', 'Tipo Rede', 'Código Ponto', 'Status Ponto', 'Hora Coleta', 'UGRHI', 'Município', 'UF', 'CLASSE', 'Sistema Hídrico', 'Tipo de Sistema Hídrico', 'Localização', 'Captação', 'Altitude', 'Latitude', 'Longitude', 'Inicio Operação', 'Fim Operação']
parametrosRelevantes = ['pH', 'Temperatura da Água', 'Sólido Dissolvido Total', 'Oxigênio Dissolvido', 'Turbidez', 'Nitrogênio-Nitrito', 'Nitrogênio-Nitrato', 'Nitrogênio Amoniacal', 'Fósforo Total', 'DBO (5, 20)', 'Coliformes Termotolerantes']
dfFiltered = df.drop(droppers, axis=1)
dfParameters = dfFiltered[dfFiltered['Parametro'].isin(parametrosRelevantes)].reset_index() 
dfParameters.sort_values(by='Data Coleta', inplace=True)

In [ ]:
parametersDfs={}

for parameter in parametrosRelevantes:
    parametersDfs[parameter] = dfParameters[dfParameters['Parametro']==parameter].reset_index()
    if parameter != 'Coloração':
        parametersDfs[parameter]['Valor'] = pd.to_numeric(parametersDfs[parameter]['Valor']) 


In [ ]:
i = 0
for parameter in parametrosRelevantes:
    if parameter != 'Coloração':
        serie = parametersDfs[parameter]
        serieValor = serie['Valor'].ewm(span=10, adjust=False).mean()
        serieValorRaw = serie['Valor']
        plt.figure(figsize=(10,10))
        plt.grid(True)
        plt.plot(serie['Data Coleta'], serieValor, c='#00035b')
        plt.plot(serie['Data Coleta'], serieValorRaw, c='#ff796c', alpha=0.5)
        plt.title(parameter)
        plt.xlabel("Data de Coleta")
        plt.ylabel(parameter)
        plt.savefig(f'plots/{i}.png')
        i += 1